In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator, ZeroCount

/usr/lib/python2.7/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
# NOTE: Make sure that the class is labeled 'target' in the data file
ruta = '/home/dataloaderusr/attributos_malos_buenos_29nov_zeros.csv'
tpot_data = pd.read_csv(ruta, sep=',', dtype=np.float64)
col_unselect = {'target', 'index', 'Call_num'}

In [3]:
features = tpot_data.drop(col_unselect,axis=1)
#features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=42)


In [4]:
# Average CV score on the training set was:0.9704658959097104
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=BernoulliNB(alpha=1.0, fit_prior=True)),
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    ZeroCount(),
    RandomForestClassifier(bootstrap=True, criterion="entropy",
                           max_features=0.4, min_samples_leaf=12,
                           min_samples_split=3, n_estimators=100)
)


In [5]:
exported_pipeline.fit(training_features, training_target)

Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))), ('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('zerocount', ZeroCount()), ('randomforestclassifier', RandomForestClas...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [6]:
results = exported_pipeline.predict(testing_features)
sum(results)

20.0

In [7]:
# save the model to disk  exported_pipeline
import pickle
filename = '/home/dataloaderusr/fraude_model_19mar.sav'
pickle.dump(exported_pipeline, open(filename, 'wb'))